## MultiIndex: uma forma de armazenar informações multidimensionais em um vetor
---

In [1]:
import numpy as np
import pandas as pd

pandas tem o tipo `MultiIndex` capaz de receber, como índice, tuplas contendo diversos valores, sendo, assim, possível armazenar mais de uma informação em um `Series`.

para isto, é antes necessário já dispor da lista de tuplas que servirá como índice mais tarde:

In [2]:
índices = [('b', 1), ('b', 0), ('c', 0), ('d', 4), ('d', 1), ('e', 0)]

em seguida, os valores e a `Series` devem ser declaradas normalmente:

In [3]:
vlr = [np.e, np.pi, np.pi**np.e, np.e/np.pi, -1/np.pi, np.sqrt(np.e)]

tudo = pd.Series(vlr, index=índices)
tudo

(b, 1)     2.718282
(b, 0)     3.141593
(c, 0)    22.459158
(d, 4)     0.865256
(d, 1)    -0.318310
(e, 0)     1.648721
dtype: float64

até agora, o que se tem é apenas tuplas em forma de índices, mas que, ainda, não é capaz de fazer o que `MultiIndex` faz. Para tal, é preciso redeclarar os índices para que este passe a ser do tipo `MultiIndex`:

In [4]:
índices = pd.MultiIndex.from_tuples(índices)

e, agora, é necessário reorganizar o vetor `tudo` com o método `.reindex()` que recebe como parâmetro a lista de tuplas:

In [5]:
tudo = tudo.reindex(índices)

agora, sim:

In [6]:
tudo

b  1     2.718282
   0     3.141593
c  0    22.459158
d  4     0.865256
   1    -0.318310
e  0     1.648721
dtype: float64

observe que o pandas organiza cada valor dos índices em hierarquias, onde cada espaço vazio corresponde ao valor na linha preenchida mais próxima.

mas, nem sempre é necessário criar um `MultiIndex` explicitamente: só em passar, através do parâmetro `index=`, uma lista contendo os arrays para construir os índices, o pandas já faz isso altomaticamente:

In [7]:
tudo = pd.Series(vlr, index=[['b', 'b', 'c', 'd', 'd', 'e'], [1, 0, 0, 4, 1, 0]])
tudo

b  1     2.718282
   0     3.141593
c  0    22.459158
d  4     0.865256
   1    -0.318310
e  0     1.648721
dtype: float64

ou mesmo, através de dicionários, onde, da mesma forma, os índices devem já ser os conjunto de arrays:

In [8]:
tudo = pd.Series(vlr, index={('b', 1): vlr[0], ('b', 0): vlr[1],
                             ('c', 0): vlr[2], ('d', 4): vlr[3],
                             ('d', 0): vlr[4], ('e', 0): vlr[5]})
tudo

b  1     2.718282
   0     3.141593
c  0    22.459158
d  4     0.865256
   0    -0.318310
e  0     1.648721
dtype: float64

este tipo de series pode ainda ser criado com os métodos `.from_arrays()`, `.from_frame()` e `.from_product()`, ou diretamente pela classe `.MultiIndex(levels=[<listas_indices>], labels=[<lista_valores>])`

especificamente, o método `.from_product()` funciona da seguinte maneira:

In [9]:
tuplas = [1, 2, 3, 4, 5, 6]
vtr = pd.Series(tuplas, pd.MultiIndex.from_product([[11, 12], ['a', 'b', 'c']]))
print(vtr)

11  a    1
    b    2
    c    3
12  a    4
    b    5
    c    6
dtype: int64


É necessário passar as informações, neste exemplo, a lista `tuplas`, seguida de `pd.Multiindex.from_product([<lista_de_índices>])`, onde deve ser apresentado uma lista com cada índice e subíndice. 
o tamanho da tupla, neste caso, precisa ser igual à multiplicação dos índices.

#### ordenação
---

é interessante observar que o `MultiIndex` não funciona se os índices não estiverem organizados:

In [10]:
índices = [('b', 1), ('c', 0), ('b', 0), ('d', 1), ('e', 0), ('d', 4)]
vlr = [np.e, np.pi, np.pi**np.e, np.e/np.pi, -1/np.pi, np.sqrt(np.e)]
tudo = pd.Series(vlr, index=índices)

índices = pd.MultiIndex.from_tuples(índices)
tudo = tudo.reindex(índices)
tudo

b  1     2.718282
c  0     3.141593
b  0    22.459158
d  1     0.865256
e  0    -0.318310
d  4     1.648721
dtype: float64

para não precisar fazer isto manualmente, pode-se usar as funções `sort_index()` ou `sortlevel()`:

In [11]:
tudo = tudo.sort_index()
tudo

b  0    22.459158
   1     2.718282
c  0     3.141593
d  1     0.865256
   4     1.648721
e  0    -0.318310
dtype: float64

#### indexing e slicing em series
---

funciona da mesma forma que para pandas:

In [12]:
tudo['b']

0    22.459158
1     2.718282
dtype: float64

In [13]:
tudo[:, 0]

b    22.459158
c     3.141593
e    -0.318310
dtype: float64

este slicing funciona da seguinte forma:
1. como tem `:`, isto significa que deve percorrer todos os primeiros valores dos índices;
2. assim, checa se estes têm o valor 0 (zero) na segunda posição;
3. se tiver, retorna o valor passado como para a series.

In [14]:
tudo['d', 4] #tudo['d'][4]

1.6487212707001282

In [15]:
tudo['b':'c']

b  0    22.459158
   1     2.718282
c  0     3.141593
dtype: float64

In [16]:
tudo[['b', 'c']]

b  0    22.459158
   1     2.718282
c  0     3.141593
dtype: float64

fancing indexing também funciona aqui:

In [17]:
tudo[tudo < 0]

e  0   -0.31831
dtype: float64

In [18]:
tudo[tudo > 2.8]

b  0    22.459158
c  0     3.141593
dtype: float64

#### em dataframmes
---

é importante ver que isto também funciona com dataframes:

In [19]:
tudo_df = pd.DataFrame({'tudo': tudo})
tudo_df

tudo
b 0  22.459158
  1   2.718282
c 0   3.141593
d 1   0.865256
  4   1.648721
e 0  -0.318310

os dataframes, desta forma, conseguem comportar muito mais informações, ficando cada vez mais semelhante a uma planilha excel, por exemplo.

é até mesmo possível adicionar informações a este dataframe:

In [20]:
tudo_df['adicionado'] = [1, 2, 3, 4, 5, 6]
tudo_df

tudo  adicionado
b 0  22.459158           1
  1   2.718282           2
c 0   3.141593           3
d 1   0.865256           4
  4   1.648721           5
e 0  -0.318310           6

#### .unstack() e .stack()
---

este método retorna a series de índices multidimensionais como um dataframe:

In [21]:
tudo_df = tudo.unstack()
tudo_df

,0,1,4
b,22.459158,2.718282,NaN
c,3.141593,NaN,NaN
d,NaN,0.865256,1.648721
e,-0.318310,NaN,NaN


o método `unstack()` ainda recebe o parâmetro `level=`, que recebe o número do nível ao qual deseja-se usar:

In [22]:
tudo_df_0 = tudo.unstack(level=0)
tudo_df_1 = tudo.unstack(level=1)

In [23]:
tudo_df_0

,b,c,d,e
0,22.459158,3.141593,NaN,-0.31831
1,2.718282,NaN,0.865256,NaN
4,NaN,NaN,1.648721,NaN


In [24]:
tudo_df_1

,0,1,4
b,22.459158,2.718282,NaN
c,3.141593,NaN,NaN
d,NaN,0.865256,1.648721
e,-0.318310,NaN,NaN


o método `.stack()` faz o contrário, transformando um dataframe em uma series de índices multidimensionais

In [25]:
tudo = tudo_df.stack()
tudo

b  0    22.459158
   1     2.718282
c  0     3.141593
d  1     0.865256
   4     1.648721
e  0    -0.318310
dtype: float64

#### nomeando os níveis
---

pode ser interessante dar nome aos níveis. isto é possível com o atributo `.index.names` que recebe um array com os nomes de cada nível:

In [26]:
tudo.index.names = ['letras', 'valores']
tudo

letras  valores
b       0          22.459158
        1           2.718282
c       0           3.141593
d       1           0.865256
        4           1.648721
e       0          -0.318310
dtype: float64

ou, pode já passar como parâmetro em `MultiIndex(names=[<lista_nomes>])`

#### em colunas
---

é importante ressaltar que estes níveis também podem ser passados para as colunas, nos dataframes:

In [27]:
col = pd.MultiIndex.from_product([['ontem', 'hoje', 'amanhã'], ['a', 'b']], names=['dia', 'ocorrência'])

df = pd.DataFrame(np.random.random((6, 6)), columns=col)
df

dia            ontem                hoje              amanhã          
ocorrência         a         b         a         b         a         b
0           0.886886  0.185055  0.844796  0.378107  0.528120  0.282434
1           0.655544  0.806760  0.116705  0.035950  0.503194  0.806363
2           0.316650  0.595503  0.717987  0.715678  0.284921  0.254851
3           0.359189  0.223585  0.586297  0.236828  0.053456  0.527179
4           0.454802  0.079668  0.966680  0.932688  0.524534  0.510351
5           0.431570  0.835878  0.306136  0.075291  0.866753  0.190780

e que os níveis podem ser passados para colunas e indices ao mesmo tempo:

In [28]:
lin = pd.MultiIndex.from_product([['manhã', 'noite'], [1, 0]], names=['momento', 'solução'])
col = pd.MultiIndex.from_product([['ontem', 'hoje', 'amanhã'], ['a', 'b']], names=['dia', 'ocorrência'])

df = pd.DataFrame(np.random.random((4, 6)), columns=col, index=lin)
df

dia                 ontem                hoje              amanhã          
ocorrência              a         b         a         b         a         b
momento solução                                                            
manhã   1        0.232649  0.768357  0.983172  0.666253  0.455210  0.959912
        0        0.746326  0.674490  0.303773  0.402837  0.634074  0.320739
noite   1        0.438667  0.362816  0.022010  0.980050  0.017480  0.302203
        0        0.897637  0.844828  0.525961  0.508599  0.196806  0.905169

#### indexing e slicing em dataframes
---

para indexing, é importante lembrar que sempre as colunas vêm primeiro; enquanto que para slicing, quem vem primeiro são as linhas.

nestes casos, os processos de indexing e slicing ocorrem nos seguintes moldes:

In [29]:
df['hoje']

ocorrência              a         b
momento solução                    
manhã   1        0.983172  0.666253
        0        0.303773  0.402837
noite   1        0.022010  0.980050
        0        0.525961  0.508599

In [30]:
df['hoje', 'a']

momento  solução
manhã    1          0.983172
         0          0.303773
noite    1          0.022010
         0          0.525961
Name: (hoje, a), dtype: float64

In [31]:
df['ontem', 'b']['manhã', 0]

0.6744900728494511

In [32]:
df['amanhã', 'a'][:, 1]

momento
manhã    0.45521
noite    0.01748
Name: (amanhã, a), dtype: float64

In [33]:
df[df > 0.67]

dia                 ontem                hoje          amanhã          
ocorrência              a         b         a        b      a         b
momento solução                                                        
manhã   1             NaN  0.768357  0.983172      NaN    NaN  0.959912
        0        0.746326  0.674490       NaN      NaN    NaN       NaN
noite   1             NaN       NaN       NaN  0.98005    NaN       NaN
        0        0.897637  0.844828       NaN      NaN    NaN  0.905169

In [34]:
df.loc[:, ('ontem', 'a')]

momento  solução
manhã    1          0.232649
         0          0.746326
noite    1          0.438667
         0          0.897637
Name: (ontem, a), dtype: float64

a atributo `.indexSlice` ajuda, ainda, no slicing, já que, no geral, esta estrutura não suporta tão processo:

In [35]:
idx = pd.IndexSlice
df.loc[idx[:, 0], idx['amanhã', :]]

dia                amanhã          
ocorrência              a         b
momento solução                    
manhã   0        0.634074  0.320739
noite   0        0.196806  0.905169

este atributo só pode ser usado com o atributo `.loc`

In [36]:
df.loc[idx[:, 1], idx[:, :]]

dia                 ontem                hoje             amanhã          
ocorrência              a         b         a         b        a         b
momento solução                                                           
manhã   1        0.232649  0.768357  0.983172  0.666253  0.45521  0.959912
noite   1        0.438667  0.362816  0.022010  0.980050  0.01748  0.302203

#### reset_index
---

este método reformula o dataframe ou series, tornando os níveis em simples colunas separadas:

In [40]:
df_flat = df.reset_index()
df_flat

dia        momento solução     ontem                hoje              amanhã  \
ocorrência                         a         b         a         b         a   
0            manhã       1  0.232649  0.768357  0.983172  0.666253  0.455210   
1            manhã       0  0.746326  0.674490  0.303773  0.402837  0.634074   
2            noite       1  0.438667  0.362816  0.022010  0.980050  0.017480   
3            noite       0  0.897637  0.844828  0.525961  0.508599  0.196806   

dia                   
ocorrência         b  
0           0.959912  
1           0.320739  
2           0.302203  
3           0.905169

ainda recebe o parâmetro `name=` que nomea a(s) nova(s) coluna(s):

#### set_index()
---

esta função faz o inverso da anterior, organizando series e dataframes com multiindexing:

In [38]:
df_flat.set_index(['momento', 'solução'])

dia                 ontem                hoje              amanhã          
ocorrência              a         b         a         b         a         b
momento solução                                                            
manhã   1        0.232649  0.768357  0.983172  0.666253  0.455210  0.959912
        0        0.746326  0.674490  0.303773  0.402837  0.634074  0.320739
noite   1        0.438667  0.362816  0.022010  0.980050  0.017480  0.302203
        0        0.897637  0.844828  0.525961  0.508599  0.196806  0.905169

observe que é necessário passar o nome para os índices.

#### cálculo de dados
---

as operações `.mean()`, `.sum(]`, `.max()`, etc., podem, normalmente, calcular os resultados dessas series e dataframes, mesmo com o muldiindexing

In [42]:
df

dia                 ontem                hoje              amanhã          
ocorrência              a         b         a         b         a         b
momento solução                                                            
manhã   1        0.232649  0.768357  0.983172  0.666253  0.455210  0.959912
        0        0.746326  0.674490  0.303773  0.402837  0.634074  0.320739
noite   1        0.438667  0.362816  0.022010  0.980050  0.017480  0.302203
        0        0.897637  0.844828  0.525961  0.508599  0.196806  0.905169

In [43]:
df.mean()

dia     ocorrência
ontem   a             0.578820
        b             0.662623
hoje    a             0.458729
        b             0.639435
amanhã  a             0.325892
        b             0.622006
dtype: float64

especificando o nível, através do parâmetro `level`, é possível calcular de um multiindex específico passando o npme do nível, neste exemplo, poderia ser `momento`. Pode, ainda, especificar o eixo através do parâmetro `axis`.